In [1]:
import pandas as pd

which_wiki =4

In [2]:
import json

# original_sentences
with open(f'new_wiki_original_sentences_{which_wiki}.txt') as f:
    original_sentences = f.readlines()
    for i in range(len(original_sentences)):
        original_sentences[i] = original_sentences[i].strip()

# ctrl-t5-3b
with open(f'ctrl-t5-3B-new-wiki-{which_wiki}-output.txt') as f:
    t5_3b = f.readlines()
    for i in range(len(t5_3b)):
        t5_3b[i] = t5_3b[i].strip()

# ctrl-t5-11b
with open(f'ctrl-t5-11B-new-wiki-{which_wiki}-output.txt') as f:
    t5_11b = f.readlines()
    for i in range(len(t5_11b)):
        t5_11b[i] = t5_11b[i].strip()

In [3]:
# make dataframe with all outputs
df = pd.DataFrame()
df['Original Sentence'] = original_sentences
df['Ctrl-T5-3b'] = t5_3b
df['Ctrl-T5-11b'] = t5_11b

In [4]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}-ctrl-t5-before-splitting.csv', index=False)

In [5]:
from string import punctuation
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def remove_tokenization_artifacts(s, src):
    stokens = s.split()
    snew = s
    for i, token in enumerate(stokens):
        if i > 0 and i < len(stokens) - 1 and token in punctuation:
            substrboth = stokens[i - 1] + token + stokens[i + 1]
            substrleft = stokens[i - 1] + token
            substright = token + stokens[i + 1]
            if substrboth in src:
                snew = snew.replace(stokens[i - 1] + " " + token + " " + stokens[i + 1], substrboth)
            elif substrleft in src:
                snew = snew.replace(stokens[i - 1] + " " + token, substrleft)
            elif substright in src:
                snew = snew.replace(token + " " + stokens[i + 1], substright)

    snew = snew.replace("''", '"')
    snew = snew.replace(" .", ".")
    snew_rest = "" if len(snew) == 1 else snew[1:]
    if len(snew) > 0:
        snew = snew[0].capitalize() + snew_rest
    snew = snew.replace("-lrb-", "(").replace("-rrb-", ")")
    snew = snew.replace("-LRB-", "(").replace("-RRB-", ")")
    return snew

def reformat_output(output, original):
    output = remove_tokenization_artifacts(output, original)
    output = " || ".join(sent_tokenize(output))
    return output

[nltk_data] Downloading package punkt to /Users/yaod_1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# reformat output with reformat_output function
for index, row in df.iterrows():
    df.loc[index, 'Ctrl-T5-3b'] = reformat_output(row['Ctrl-T5-3b'], row['Original Sentence'])
    df.loc[index, 'Ctrl-T5-11b'] = reformat_output(row['Ctrl-T5-11b'], row['Original Sentence'])

In [7]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}-ctrl-t5.csv', index=False)